In [1]:
import os
import numpy as np
import pandas as pd

import rocks

rocks.set_log_level("error")

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import figure_setup as fs

from figure_mask import compute_mask, print_statistics, compute_mask_spin

In [2]:
# G12 line in G1G2 space
def G12_to_G1G2(g12):
    G1_a, G2_a = 0.0, 0.53513350
    G1_b, G2_b = 0.84293649, -0.53513350

    G1 = G1_a + g12 * G1_b
    G2 = G2_a + g12 * G2_b

    return G1, G2


# Simple 1D Gaussian
def gauss(x, m, mu, sigma):
    return m * np.exp(-((x - mu) ** 2) / (2 * sigma**2))


# Two 1D Gaussians
def double_gauss(x, m1, mu1, sigma1, m2, mu2, sigma2):
    g1 = gauss(x, m1, mu1, sigma1)
    g2 = gauss(x, m2, mu2, sigma2)
    return g1 + g2

# Get ZTF fit

In [3]:
# Local Configuration
data_fink = "../"
bft_file = os.path.join(data_fink, "data", "ssoBFT-latest.parquet")

In [4]:
# ZTF filters
filters = {"1": "g", "2": "r"}

# S_color = {'g': -0.3928, 'r':0.2913}
# sun_color = {'g': -0.3044, 'r':0.1903}

# color_C = -(sun_color['g']-sun_color['r'])
# color_S = -(S_color['g']-S_color['r'])


# mag_to_V ={
#         1: -0.2833,  # ZTF g
#         2: 0.1777,   # ZTF r
#         3: 0.4388,   # ATLAS orange
#         4: -0.0986   # ATLAS cyan
#     }

fink_colors = ["#15284F", "#F5622E"]

In [5]:
data = pd.read_parquet(os.path.join(data_fink, "data", "ztf", "sso_ZTF.parquet"))

In [6]:
cols = [
    "sso_number",
    "sso_name",
    "sso_class",
    "orbital_elements.semi_major_axis.value",
    "orbital_elements.eccentricity.value",
    "orbital_elements.inclination.value",
    "orbital_elements.node_longitude.value",
    "orbital_elements.perihelion_argument.value",
    "orbital_elements.mean_anomaly.value",
    "orbital_elements.mean_motion.value",
    "family.family_number",
    "family.family_name",
    "tisserand_parameters.Jupiter.value",
    "albedo.value",
    "absolute_magnitude.value",
    "diameter.value",
    "taxonomy.class",
    "taxonomy.complex",
    "taxonomy.waverange",
    "taxonomy.scheme",
    "taxonomy.technique",
    "colors.g-r.color.value",
    "colors.g-r.color.error.min",
    "colors.g-r.color.error.max",
    "colors.g-r.facility",
    "colors.g-r.observer",
    "colors.g-r.epoch",
    "colors.g-r.delta_time",
    "colors.g-r.id_filter_1",
    "colors.g-r.id_filter_2",
    "colors.g-r.phot_sys",
    "colors.g-r.technique",
    "spins.1.obliquity",
    "spins.1.RA0.value",
    "spins.1.DEC0.value",
    "spins.1.long.value",
    "spins.1.lat.value",
    "spins.1.technique",
    "spins.2.obliquity",
    "spins.2.RA0.value",
    "spins.2.DEC0.value",
    "spins.2.long.value",
    "spins.2.lat.value",
    "spins.2.technique",
    "spins.3.obliquity",
    "spins.3.RA0.value",
    "spins.3.DEC0.value",
    "spins.3.long.value",
    "spins.3.lat.value",
    "spins.3.technique",
    "spins.4.obliquity",
    "spins.4.RA0.value",
    "spins.4.DEC0.value",
    "spins.4.long.value",
    "spins.4.lat.value",
    "spins.4.technique",
]
bft = pd.read_parquet(bft_file, columns=cols)


data = data.merge(bft, left_on="name", right_on="sso_name", how="left")

In [7]:
len(data)

122675

# Compare results

In [8]:
data.HG1G2_status.value_counts()

 2.0    113644
 4.0      6886
 1.0      1320
 3.0       521
-2.0       301
 0.0         3
Name: HG1G2_status, dtype: int64

In [9]:
mask_SHG1G2_union = compute_mask(data, model='SHG1G2', kind='union')
mask_HG1G2_union = compute_mask(data, model='HG1G2', kind='union')
mask_HG_union = compute_mask(data, model='HG', kind='union')

mask_SHG1G2_inter = compute_mask(data, model='SHG1G2', kind='inter')
mask_HG1G2_inter = compute_mask(data, model='HG1G2', kind='inter')
mask_HG_inter = compute_mask(data, model='HG', kind='inter')

maskFINK_inter = compute_mask(data, R_min=0.305, model='FINK', kind='inter')
maskFINK_union = compute_mask(data, R_min=0.305, model='FINK', kind='union')

for model in ['HG', 'HG1G2', 'SHG1G2', 'FINK']:
    print_statistics(data, model)
    print()

  All data       : 122675  (100.00%)
  Mask HG (g∩r) : 120023  ( 97.84%)
  Mask HG (gUr) : 120024  ( 97.84%)

  All data       : 122675  (100.00%)
  Mask HG1G2 (g∩r) :  47177  ( 38.46%)
  Mask HG1G2 (gUr) :  85363  ( 69.58%)

  All data       : 122675  (100.00%)
  Mask SHG1G2 (g∩r) :  64279  ( 52.40%)
  Mask SHG1G2 (gUr) :  97579  ( 79.54%)

  All data       : 122675  (100.00%)
  Mask FINK (g∩r) :  63092  ( 51.43%)
  Mask FINK (gUr) :  95593  ( 77.92%)



In [10]:
maskFINK = maskFINK_union
mask_HG1G2 = mask_HG1G2_inter
mask_SHG1G2 = mask_SHG1G2_inter

## Figure RMS

In [11]:
# VERSION 3x2 with ATLAS
fig, ax = plt.subplots(
    2,
    3,
    figsize=fs.figsize(0.5),
    gridspec_kw={
        "hspace": 0.02,
        "wspace": 0.02,
        "top": 0.98,
        "bottom": 0.065,
        "left": 0.095,
        "right": 0.98,
    },
)

r = [0, 0.5]
b = 50
vmin = 0.99
vmax = 1000

# --------------------------------------------------------------------------------
x = data.loc[mask_HG1G2, "HG_rms_g"]
y = data.loc[mask_HG1G2, "HG1G2_rms_g"]
_, _, _, im_g = ax[0, 0].hist2d(
    x,
    y,
    range=[r, r],
    bins=b,
    norm="log",
    cmap="Blues",
    rasterized=True,
    vmin=vmin,
    vmax=vmax,
)

axinsg = inset_axes(
    ax[0, 0],
    width="50%",  # width: 50% of parent_bbox width
    height="5%",  # height: 5%
    loc="upper center",
)
cbar_g = fig.colorbar(im_g, cax=axinsg, orientation="horizontal")
cbar_g.ax.set_xticks([1, 1000])
cbar_g.ax.set_xticklabels(["1", "1000"], fontsize="small")

ax[0, 0].set_ylabel("RMS in g")
ax[0, 0].text(0.95, 0.05, "$\\texttt{HG}$", ha="right", transform=ax[0, 0].transAxes)
ax[0, 0].text(
    0.1,
    0.95,
    "$\\texttt{HG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$",
    va="top",
    ha="center",
    rotation=90,
    transform=ax[0, 0].transAxes,
)


x = data.loc[mask_HG1G2, "HG_rms_r"]
y = data.loc[mask_HG1G2, "HG1G2_rms_r"]
_, _, _, im_r = ax[1, 0].hist2d(
    x,
    y,
    range=[r, r],
    bins=b,
    norm="log",
    cmap="Oranges",
    rasterized=True,
    vmin=vmin,
    vmax=vmax,
)


axinsr = inset_axes(
    ax[1, 0],
    width="50%",  # width: 50% of parent_bbox width
    height="5%",  # height: 5%
    loc="upper center",
)
cbar_r = fig.colorbar(im_r, cax=axinsr, orientation="horizontal")
cbar_r.ax.set_xticks([1, 1000])
cbar_r.ax.set_xticklabels(["1", "1000"], fontsize="small")

ax[1, 0].set_ylabel("RMS in r")
ax[1, 0].text(0.95, 0.05, "$\\texttt{HG}$", ha="right", transform=ax[1, 0].transAxes)
ax[1, 0].text(
    0.1,
    0.95,
    "$\\texttt{HG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$",
    va="top",
    ha="center",
    rotation=90,
    transform=ax[1, 0].transAxes,
)

# --------------------------------------------------------------------------------
x = data.loc[mask_SHG1G2, "HG_rms_g"]
y = data.loc[mask_SHG1G2, "SHG1G2_rms_g"]
ax[0, 1].hist2d(
    x,
    y,
    range=[r, r],
    bins=b,
    norm="log",
    cmap="Blues",
    rasterized=True,
    vmin=vmin,
    vmax=vmax,
)
ax[0, 1].text(0.95, 0.05, "$\\texttt{HG}$", ha="right", transform=ax[0, 1].transAxes)
ax[0, 1].text(
    0.1,
    0.95,
    "$\\texttt{sHG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$",
    va="top",
    ha="center",
    rotation=90,
    transform=ax[0, 1].transAxes,
)

x = data.loc[mask_SHG1G2, "HG_rms_r"]
y = data.loc[mask_SHG1G2, "SHG1G2_rms_r"]
ax[1, 1].hist2d(
    x,
    y,
    range=[r, r],
    bins=b,
    norm="log",
    cmap="Oranges",
    rasterized=True,
    vmin=vmin,
    vmax=vmax,
)
ax[1, 1].text(0.95, 0.05, "$\\texttt{HG}$", ha="right", transform=ax[1, 1].transAxes)
ax[1, 1].text(
    0.1,
    0.95,
    "$\\texttt{sHG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$",
    va="top",
    ha="center",
    rotation=90,
    transform=ax[1, 1].transAxes,
)

# --------------------------------------------------------------------------------
mm = mask_HG1G2 & mask_SHG1G2
x = data.loc[mm, "HG1G2_rms_g"]
y = data.loc[mm, "SHG1G2_rms_g"]
ax[0, 2].hist2d(
    x,
    y,
    range=[r, r],
    bins=b,
    norm="log",
    cmap="Blues",
    rasterized=True,
    vmin=vmin,
    vmax=vmax,
)
ax[0, 2].text(
    0.95,
    0.05,
    "$\\texttt{HG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$",
    ha="right",
    transform=ax[0, 2].transAxes,
)
ax[0, 2].text(
    0.1,
    0.95,
    "$\\texttt{sHG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$",
    va="top",
    ha="center",
    rotation=90,
    transform=ax[0, 2].transAxes,
)

x = data.loc[mm, "HG1G2_rms_r"]
y = data.loc[mm, "SHG1G2_rms_r"]
ax[1, 2].hist2d(
    x,
    y,
    range=[r, r],
    bins=b,
    norm="log",
    cmap="Oranges",
    rasterized=True,
    vmin=vmin,
    vmax=vmax,
)
ax[1, 2].text(
    0.95,
    0.05,
    "$\\texttt{HG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$",
    ha="right",
    transform=ax[1, 2].transAxes,
)
ax[1, 2].text(
    0.1,
    0.95,
    "$\\texttt{sHG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$",
    va="top",
    ha="center",
    rotation=90,
    transform=ax[1, 2].transAxes,
)

# --------------------------------------------------------------------------------
for a in ax[:, 1:].ravel():
    a.set_yticklabels("")
for a in ax[0, :].ravel():
    a.set_xticklabels("")
for a in ax.ravel():
    a.set_aspect("equal")
    
xx = np.linspace(0, 1, num=10)
for a in ax[:].ravel():
    a.set_xlim(r[0], r[1])
    a.set_ylim(r[0], r[1])
    a.plot(xx, xx, linestyle="dashed", color="grey")
    a.grid()
    a.set_aspect("equal")

# --------------------------------------------------------------------------------
fig.savefig(
    os.path.join(data_fink, "gfx", "article", "rms_comparison_masked.png"),
    facecolor="white",
)

fig.savefig(
    os.path.join(data_fink, "gfx", "article", "rms_comparison_masked.pgf"),
)
plt.close()

below is exploratory

# Lambda vs RMS

In [15]:
def overplot_running_mean(
    x,
    y,
    step,
    axe,
    show_std=True,
    lower=None,
    upper=None,
    color="red",
    label="Running mean",
):
    """
    Overplots the running mean of y values with error bars on a given axis.

    Parameters:
    - x (array-like): The x values.
    - y (array-like): The y values.
    - step (float): The step size for calculating the running mean.
    - axe (matplotlib.axes.Axes): The axis to plot on.
    - show_std (bool, optional): Whether to show the standard deviation as error bars. Default is True.
    - lower (float, optional): The lower limit for the error bars. Default is None.
    - upper (float, optional): The upper limit for the error bars. Default is None.
    - color (str, optional): The color of the plot. Default is 'red'.
    - label (str, optional): The label for the plot. Default is 'Running mean'.
    """

    hist_average = pd.DataFrame()
    loc = 0
    xmax = x.max()
    xx = x.min()

    while xx <= xmax:
        cond = (xx <= x) & (x < xx + step)
        yy = y[cond]
        if len(y) > 0:
            hist_average.loc[loc, "x"] = xx + step / 2
            hist_average.loc[loc, "y"] = np.mean(yy)
            hist_average.loc[loc, "yerr"] = np.std(yy)
            loc += 1
        xx += step
    hist_average

    axe.plot(
        hist_average["x"],
        hist_average["y"],
        alpha=1.0,
        marker="o",
        markersize=4,
        color=color,
        label="Running mean",
    )

    if show_std:
        lower_y = hist_average["y"] - hist_average["yerr"]
        upper_y = hist_average["y"] + hist_average["yerr"]

        if lower is not None:
            lower_y = [lower if ll < lower else ll for ll in lower_y]
        if upper is not None:
            upper_y = [upper if uppeuur_y > upper else uu for uu in upper_y]

        axe.fill_between(
            hist_average["x"],
            y1=lower_y,
            y2=upper_y,
            alpha=0.3,
            color=color,
            rasterized=True,
            label="Standard deviation",

        )

In [17]:
fig, ax = plt.subplots(
    1,
    3,
    figsize=fs.figsize(1, aspect=3.2),
    sharey=True,
    gridspec_kw={
        "hspace": 0.02,
        "wspace": 0.02,
        "top": 0.98,
        "bottom": 0.14,
        "left": 0.06,
        "right": 0.99,
    },
)

r_rms = [0, 0.2]

r_Lambda = [0, 90]
r_obliquity = [0, 180]
r_R = [0.3, 1]

b_rms = 20
b_Lambda = 18
b_obliquity = 18
b_R = 28

vmin = 1
vmax = 1000

# --------------------------------------------------------------------------------
# --------------------------------------------------------------------------------
# RMS vs Lambda
xarr = np.degrees(
    np.arccos(data.loc[maskFINK, "SHG1G2_min_cos_lambda"])
    - np.arccos(data.loc[maskFINK, "SHG1G2_max_cos_lambda"])
)
yarr = data.loc[maskFINK, "HG1G2_rms"] - data.loc[maskFINK, "SHG1G2_rms"]

_, _, _, im_Lambda = ax[0].hist2d(
    xarr,
    yarr,
    range=[r_Lambda, r_rms],
    bins=[b_Lambda, b_rms],
    norm="log",
    cmap="Blues",
    rasterized=True,
    vmin=vmin,
    vmax=vmax,
)

# Running average
overplot_running_mean(
    xarr,
    yarr,
    5,
    ax[0],
    show_std=True,
    lower=0,
)


# --------------------------------------------------------------------------------
# --------------------------------------------------------------------------------
# RMS vs obliquity
xarr = data.loc[maskFINK, "SHG1G2_obliquity"]
yarr = data.loc[maskFINK, "HG1G2_rms"] - data.loc[maskFINK, "SHG1G2_rms"]

_, _, _, im_obliquity = ax[1].hist2d(
    xarr,
    yarr,
    range=[r_obliquity, r_rms],
    bins=[b_obliquity, b_rms],
    norm="log",
    cmap="Blues",
    rasterized=True,
    vmin=vmin,
    vmax=vmax,
)

# Running average
overplot_running_mean(
    xarr,
    yarr,
    10,
    ax[1],
    show_std=True,
    lower=0,
)


# --------------------------------------------------------------------------------
# --------------------------------------------------------------------------------
# --------------------------------------------------------------------------------
# RMS vs Oblateness
xarr = data.loc[maskFINK, "n_obs"]
xarr = data.loc[maskFINK, "max_phase"] - data.loc[maskFINK, "min_phase"]
xarr = data.loc[maskFINK, "SHG1G2_R"]
yarr = data.loc[maskFINK, "HG1G2_rms"] - data.loc[maskFINK, "SHG1G2_rms"]

_, _, _, im_R = ax[2].hist2d(
    xarr,
    yarr,
    range=[r_R, r_rms],
    bins=[b_R, b_rms],
    norm="log",
    cmap="Blues",
    rasterized=True,
    vmin=vmin,
    vmax=vmax,
)

# Running average
overplot_running_mean(
    xarr,
    yarr,
    0.05,
    ax[2],
    show_std=True,
    lower=0,
)

ax[2].legend(loc="upper right")

# --------------------------------------------------------------------------------
# Color bar
axinsr = inset_axes(
    ax[0],
    width="50%",  # width: 50% of parent_bbox width
    height="5%",  # height: 5%
    loc="upper left",
)
cbar_r = fig.colorbar(im_R, cax=axinsr, orientation="horizontal")
cbar_r.ax.set_xticks([1, 10, 100, 1000])
cbar_r.ax.set_xticklabels(["1", "10", "100", "1000"], fontsize="small")


# --------------------------------------------------------------------------------
# Axes
# ax[0].set_ylabel("$\\Delta$ RMS ()")
ax[0].set_ylabel(
    "$\\Delta$ RMS ($\\texttt{HG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$ - $\\texttt{sHG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$, mag.)"
)

ax[0].set_xlabel("$\\Delta \\Lambda$ ($^{\\circ}$)")
ax[1].set_xlabel("Obliquity ($^{\\circ}$)")
ax[2].set_xlabel("Oblateness R")
ax[0].set_ylim(0, 0.2)

# Set y-axis tick interval
ax[0].set_yticks(np.arange(0, 0.21, 0.05))

# --------------------------------------------------------------------------------
fig.savefig(
    os.path.join(data_fink, "gfx", "article", "rms_vs_Lambda.pgf")
)
fig.savefig(
    os.path.join(data_fink, "gfx", "article", "rms_vs_Lambda.png"), facecolor="white", dpi=180
)

/tmp/ipykernel_12626/3489312204.py:123: UserWarning: Legend does not support handles for QuadMesh instances.
See: https://matplotlib.org/stable/tutorials/intermediate/legend_guide.html#implementing-a-custom-legend-handler
  ax[2].legend(loc="upper right")


In [ ]:
dRMS = data["HG1G2_rms"] - data["SHG1G2_rms"]
dRMS
cond = maskFINK & (dRMS > 0.20)
data[cond]

In [ ]:
data.columns[:60]